# Init

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
import tensorflow_ranking as tfr
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0
    
# 逐个读取feature_engineered_training_chunk{i}并上下拼接到一个df
base_path = 'D:/Table/P5/DM-AS2/Data_featured_menghan/'
file_pattern = 'best_feature_engineered_training_chunk_{}.csv'
for i in range(10):
    df_chunk = pd.read_csv(base_path + file_pattern.format(i))
    df_chunk['score'] = df_chunk.apply(assign_scores, axis=1)
    if i == 0:
        df = df_chunk
    else:
        df = pd.concat([df, df_chunk], axis=0)
df.head()

In [ ]:
# 导出df的前100行生成一个CSV文件
df.head(500).to_csv('df_top_500.csv', index=False)

# Test1

In [ ]:
import tensorflow as tf
import tensorflow_ranking as tfr
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import ndcg_score

# 假设df已经加载并包含数据
# df = pd.read_csv('your_data.csv')

# 获取唯一的查询ID
unique_ids = df['srch_id'].unique()

# 划分训练集和测试集的查询ID
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42, shuffle=True)

# 根据train_ids划分训练集和验证集的查询ID
train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42, shuffle=True)

# 根据查询ID过滤数据
train_df = df[df['srch_id'].isin(train_ids)]
val_df = df[df['srch_id'].isin(val_ids)]
test_df = df[df['srch_id'].isin(test_ids)]

# 特征列，排除不需要的列
feature_columns = [col for col in df.columns if col not in ['srch_id', 'date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd']]

# 提取特征和标签
X_train = train_df[feature_columns].astype(np.float32)
y_train = train_df['score'].astype(np.float32).values
X_val = val_df[feature_columns].astype(np.float32)
y_val = val_df['score'].astype(np.float32).values
X_test = test_df[feature_columns].astype(np.float32)
y_test = test_df['score'].astype(np.float32).values

# 标准化特征数据
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# 转换为TensorFlow Dataset格式
def make_dataset(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=len(y)).batch(batch_size)
    return dataset


# 创建 ApproxNDCGLoss 对象
loss = tfr.keras.losses.ApproxNDCGLoss(
    reduction=tf.losses.Reduction.AUTO,
    lambda_weight=tfr.keras.losses.DCGLambdaWeight(topn=5),  # 确定学习NDCG@5
    temperature=0.1,
    ragged=False
)

# 构建DeepRank模型
def build_model(input_shape):
    input_layer = Input(shape=(input_shape,))
    dense_layer = Dense(256, activation='relu')(input_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(128, activation='relu')(dropout_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(64, activation='relu')(dropout_layer)
    output_layer = Dense(1, activation='linear')(dense_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss)
    return model

# 构建并训练模型
input_shape = X_train.shape[1]
model = build_model(input_shape)

# 创建训练、验证和测试数据集
train_dataset = make_dataset(X_train, y_train, batch_size=32)
val_dataset = make_dataset(X_val, y_val, batch_size=32)

# 查看训练数据集中的一个批次
for X, y in train_dataset.take(1):
    X_batch = X.numpy()
    y_batch = y.numpy()
    X_shape = X.shape
    y_shape = y.shape

print("Feature batch (X):", X_shape)
print("Feature batch data (X):", X_batch)
print("Target batch (y):", y_shape)
print("Target batch data (y):", y_batch)




## model fit

In [ ]:
model.fit(train_dataset, epochs=5, validation_data=val_dataset)

# 对测试集进行预测和评估
test_pred = model.predict(X_test).flatten()
test_df['predictions'] = test_pred
test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = test_df.groupby('srch_id')
ndcg_scores = []

for name, group in grouped:
    group = group.sort_values('predictions', ascending=False)
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score@5: {average_ndcg}")

# 打印模型总结
model.summary()

# Test2

In [ ]:
import tensorflow as tf
import tensorflow_ranking as tfr
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import ndcg_score

# 假设 df 已经加载并包含数据
# df = pd.read_csv('your_data.csv')

# 获取唯一的查询ID
unique_ids = df['srch_id'].unique()

# 划分训练集和测试集的查询ID
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42, shuffle=True)

# 根据 train_ids 划分训练集和验证集的查询ID
train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42, shuffle=True)

# 根据查询ID过滤数据
train_df = df[df['srch_id'].isin(train_ids)]
val_df = df[df['srch_id'].isin(val_ids)]
test_df = df[df['srch_id'].isin(test_ids)]

# 特征列，排除不需要的列
feature_columns = [col for col in df.columns if col not in ['srch_id', 'date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd']]

# 标准化特征数据
scaler = StandardScaler()
train_df[feature_columns] = scaler.fit_transform(train_df[feature_columns])
val_df[feature_columns] = scaler.transform(val_df[feature_columns])
test_df[feature_columns] = scaler.transform(test_df[feature_columns])

def generator(dataframe):
    grouped = dataframe.groupby('srch_id')
    for name, group in grouped:
        X = group[feature_columns].values.astype(np.float32)
        y = group['score'].values.astype(np.float32)
        yield X, y[:, np.newaxis]  # 确保 y 是二维的

def make_dataset(dataframe, batch_size):
    dataset = tf.data.Dataset.from_generator(
        lambda: generator(dataframe),
        output_signature=(
            tf.TensorSpec(shape=(None, len(feature_columns)), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 1), dtype=tf.float32)  # 确保 y 的形状是 (None, 1)
        )
    )
    return dataset.batch(batch_size)

# 创建训练数据集
batch_size = 32
train_dataset = make_dataset(train_df, batch_size).shuffle(buffer_size=len(train_df['srch_id'].unique()))
val_dataset = make_dataset(val_df, batch_size).shuffle(buffer_size=len(val_df['srch_id'].unique()))
test_dataset = make_dataset(test_df, batch_size)

# 查看训练数据集中的一个批次
for X, y in train_dataset.take(1):
    X_batch = X.numpy()
    y_batch = y.numpy()
    X_shape = X.shape
    y_shape = y.shape

print("Feature batch (X):", X_shape)
print("Feature batch data (X):", X_batch)
print("Target batch (y):", y_shape)
print("Target batch data (y):", y_batch)

## model fit

In [ ]:

# 创建 ApproxNDCGLoss 对象
loss = tfr.keras.losses.ApproxNDCGLoss(
    reduction=tf.losses.Reduction.AUTO,
    lambda_weight=tfr.keras.losses.DCGLambdaWeight(topn=5),  # 确定学习NDCG@5
    temperature=0.1,
    ragged=False
)

# 构建模型
input_shape = len(feature_columns)
def build_model(input_shape):
    input_layer = Input(shape=(input_shape,))
    dense_layer = Dense(256, activation='relu')(input_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(128, activation='relu')(dropout_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(64, activation='relu')(dropout_layer)
    output_layer = Dense(1, activation='linear')(dense_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss)
    return model

# 构建并训练模型
model = build_model(input_shape)
model.fit(train_dataset, epochs=5, validation_data=val_dataset)


# 对测试集进行预测和评估
test_pred = model.predict(test_dataset)
test_pred = np.concatenate(test_pred, axis=0)

# 将预测结果与实际值进行对比，计算 NDCG@5
test_scores = []
test_groups = []

for _, y in test_dataset:
    test_scores.extend(y.numpy().flatten())  # 将y展平为一维数组
    test_groups.append(len(y))

ndcg_scores = []

start = 0
for group_size in test_groups:
    end = start + group_size
    true_relevance = test_scores[start:end]
    pred_relevance = test_pred[start:end]
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [pred_relevance], k=5))
    start = end

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score@5: {average_ndcg}")

# 打印模型总结
model.summary()

# TEst3

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 获取唯一的查询ID
unique_ids = df['srch_id'].unique()

# 划分训练集和测试集的查询ID
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42, shuffle=True)

# 根据 train_ids 划分训练集和验证集的查询ID
train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42, shuffle=True)

# 根据查询ID过滤数据
train_df = df[df['srch_id'].isin(train_ids)]
val_df = df[df['srch_id'].isin(val_ids)]
test_df = df[df['srch_id'].isin(test_ids)]

# 特征列，排除不需要的列
feature_columns = [col for col in df.columns if col not in ['srch_id', 'date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd']]

# 标准化特征数据
scaler = StandardScaler()
train_df[feature_columns] = scaler.fit_transform(train_df[feature_columns])
val_df[feature_columns] = scaler.transform(val_df[feature_columns])
test_df[feature_columns] = scaler.transform(test_df[feature_columns])

def generator(dataframe):
    grouped = dataframe.groupby('srch_id')
    for name, group in grouped:
        X = group[feature_columns].values.astype(np.float32)
        y = group['score'].values.astype(np.float32)
        yield tf.constant(X), tf.constant(y[:, np.newaxis])

def make_dataset(dataframe, batch_size):
    dataset = tf.data.Dataset.from_generator(
        lambda: generator(dataframe),
        output_signature=(
            tf.TensorSpec(shape=(None, len(feature_columns)), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 1), dtype=tf.float32)
        )
    )
    return dataset.padded_batch(batch_size, padded_shapes=([None, len(feature_columns)], [None, 1]))

# 创建训练、验证和测试数据集
batch_size = 32
train_dataset = make_dataset(train_df, batch_size).shuffle(buffer_size=len(train_df['srch_id'].unique()))
val_dataset = make_dataset(val_df, batch_size).shuffle(buffer_size=len(val_df['srch_id'].unique()))
test_dataset = make_dataset(test_df, batch_size)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout, Reshape
from tensorflow.keras.models import Model
import tensorflow_ranking as tfr

# 创建 ApproxNDCGLoss 对象
loss = tfr.keras.losses.ApproxNDCGLoss(
    reduction=tf.losses.Reduction.AUTO,
    lambda_weight=tfr.keras.losses.DCGLambdaWeight(topn=5),  # 确定学习NDCG@5
    temperature=0.1,
    ragged=False  # 不使用 ragged=True
)

# 构建模型
def build_model(input_shape):
    input_layer = Input(shape=(None, len(feature_columns)), dtype=tf.float32)
    dense_layer = Dense(256, activation='relu')(input_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(128, activation='relu')(dropout_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(64, activation='relu')(dropout_layer)
    output_layer = Dense(1, activation='linear')(dense_layer)
    output_layer = Reshape((-1, 1, 1))(output_layer)  # 调整输出形状为 [batch_size, list_size, 1, 1]
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss)
    return model

# 构建并训练模型
input_shape = (None, len(feature_columns))
model = build_model(input_shape)
model.fit(train_dataset, epochs=5, validation_data=val_dataset)

# 对测试集进行预测和评估
test_pred = model.predict(test_dataset)
test_pred = np.concatenate([pred.flatten() for pred in test_pred], axis=0)

# 将预测结果与实际值进行对比，计算 NDCG@5
test_scores = []
test_groups = []

for _, y in test_dataset:
    test_scores.extend(y.numpy().flatten())  # 将y展平为一维数组
    test_groups.append(len(y))

ndcg_scores = []

start = 0
for group_size in test_groups:
    end = start + group_size
    true_relevance = test_scores[start:end]
    pred_relevance = test_pred[start:end]
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [pred_relevance], k=5))
    start = end

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score@5: {average_ndcg}")

# 打印模型总结
model.summary()


# Test4

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
import tensorflow_ranking as tfr
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### pre

In [ ]:
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

def expand_to_fixed_length(group, length=50):
    if len(group) < length:
        # 创建一个填充 DataFrame，除了 srch_id 之外其他值均为0
        padding = pd.DataFrame(0, index=np.arange(length - len(group)), columns=group.columns)
        padding['srch_id'] = group['srch_id'].iloc[0]
        group = pd.concat([group, padding], ignore_index=True)
    return group

# 读取和处理数据
base_path = 'D:/Table/P5/DM-AS2/Data_featured_menghan/'
file_pattern = 'best_feature_engineered_training_chunk_{}.csv'
df_list = []
for i in range(1):  # 修改为实际的文件数量
    df_chunk = pd.read_csv(base_path + file_pattern.format(i))
    df_chunk['score'] = df_chunk.apply(assign_scores, axis=1)
    df_list.append(df_chunk)

# 合并所有数据
df = pd.concat(df_list, axis=0)

# 扩展每个 srch_id 的数据到50条
expanded_df = df.groupby('srch_id').apply(expand_to_fixed_length, length=50).reset_index(drop=True)

# 特征列，排除不需要的列
feature_columns = [col for col in expanded_df.columns if col not in ['srch_id', 'date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd']]

# 标准化特征数据
scaler = StandardScaler()
expanded_df[feature_columns] = scaler.fit_transform(expanded_df[feature_columns])

# 填充 NaN 值为 0
expanded_df[feature_columns] = expanded_df[feature_columns].fillna(0)

# 检查填充后是否还有 NaN 值
nan_count_after_filling = expanded_df[feature_columns].isna().sum().sum()
print(f'Number of NaN values after filling: {nan_count_after_filling}')

# 确认处理后的数据长度一致
print(expanded_df.groupby('srch_id').size().value_counts())

In [ ]:
"""
import pandas as pd

# 假设 df 是原始数据 DataFrame
# 统计每个 srch_id 的数据长度
srch_id_length_distribution = df.groupby('srch_id').size()

# 统计不同长度的分布情况
length_distribution = srch_id_length_distribution.value_counts().sort_index()

# 打印结果
print(length_distribution)
"""

In [ ]:
"""
# 统计每个 srch_id 的数据条数
srch_id_counts = df['srch_id'].value_counts()

# 过滤掉数据量小于15的 srch_id
valid_srch_ids = srch_id_counts[srch_id_counts >= 15].index
filtered_df = df[df['srch_id'].isin(valid_srch_ids)]

# 对于每个 srch_id，按照 score 值排序，并保留前15条记录
processed_df = filtered_df.groupby('srch_id').apply(lambda x: x.nlargest(15, 'score')).reset_index(drop=True)

# 特征列，排除不需要的列
feature_columns = [col for col in processed_df.columns if col not in ['srch_id', 'date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd']]

# 标准化特征数据
scaler = StandardScaler()
processed_df[feature_columns] = scaler.fit_transform(processed_df[feature_columns])

# 确认处理后的数据长度一致
print(processed_df.groupby('srch_id').size().value_counts())
"""


### 数据生成和批处理

In [ ]:
def generator(dataframe):
    grouped = dataframe.groupby('srch_id')
    for name, group in grouped:
        X = group[feature_columns].values.astype(np.float32)
        y = group['score'].values.astype(np.float32)
        yield X, y

def make_dataset(dataframe, batch_size):
    dataset = tf.data.Dataset.from_generator(
        lambda: generator(dataframe),
        output_signature=(
            tf.TensorSpec(shape=(50, len(feature_columns)), dtype=tf.float32),
            tf.TensorSpec(shape=(50,), dtype=tf.float32)
        )
    )
    return dataset.batch(batch_size)

# 获取唯一的查询ID
unique_ids = expanded_df['srch_id'].unique()

# 划分训练集和测试集的查询ID
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42, shuffle=True)

# 根据 train_ids 划分训练集和验证集的查询ID
train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42, shuffle=True)

# 根据查询ID过滤数据
train_df = expanded_df[expanded_df['srch_id'].isin(train_ids)]
val_df = expanded_df[expanded_df['srch_id'].isin(val_ids)]
test_df = expanded_df[expanded_df['srch_id'].isin(test_ids)]

# 创建训练、验证和测试数据集
batch_size = 32
train_dataset = make_dataset(train_df, batch_size).shuffle(buffer_size=len(train_df['srch_id'].unique()))
val_dataset = make_dataset(val_df, batch_size).shuffle(buffer_size=len(val_df['srch_id'].unique()))
test_dataset = make_dataset(test_df, batch_size)

# 查看训练数据集中的一个批次
for X, y in train_dataset.take(1):
    X_shape = X.shape
    y_shape = y.shape
    X_batch = X.numpy()
    y_batch = y.numpy()

print("Feature batch (X) shape:", X_shape)
print("Target batch (y) shape:", y_shape)
print("Feature batch (X) data:", X_batch)
print("Target batch (y) data:", y_batch)


### 数据处理检测

In [ ]:
print(expanded_df.isna().sum())


In [ ]:
# 标准化特征数据
scaler = StandardScaler()
expanded_df[feature_columns] = scaler.fit_transform(expanded_df[feature_columns])

# 检查标准化后是否有 NaN 值
print(expanded_df[feature_columns].isna().sum().sum())


### modeling

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout, Reshape
from tensorflow.keras.models import Model
import tensorflow_ranking as tfr

# 创建 ApproxNDCGLoss 对象
loss = tfr.keras.losses.ApproxNDCGLoss(
    reduction=tf.losses.Reduction.AUTO,
    lambda_weight=tfr.keras.losses.DCGLambdaWeight(topn=5),  # 确定学习NDCG@5
    temperature=0.1,
    ragged=False  # 不使用 ragged=True
)

# 构建模型
def build_model(input_shape):
    input_layer = Input(shape=input_shape)
    dense_layer = Dense(256, activation='relu')(input_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(128, activation='relu')(dropout_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(64, activation='relu')(dropout_layer)
    output_layer = Dense(1, activation='linear')(dense_layer)
    output_layer = tf.squeeze(output_layer, axis=-1)  # 调整输出形状为 [batch_size, list_size]
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss)
    return model

# 构建并训练模型
input_shape = (50, len(feature_columns))
model = build_model(input_shape)
model.fit(train_dataset, epochs=5, validation_data=val_dataset)

In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score

# 对测试集进行预测和评估
test_pred = model.predict(test_dataset)
test_pred = np.concatenate(test_pred, axis=0)

# 将预测结果与实际值进行对比，计算 NDCG@5
test_scores = []
test_groups = []

# 提取实际评分和 srch_id
for X, y in test_dataset:
    test_scores.extend(y.numpy().flatten())  # 将y展平为一维数组
    test_groups.extend([srch_id for srch_id in X.numpy()[:, 0, 0]])  # 假设 srch_id 是特征中的第一列

# 确保 test_pred 和 test_scores 的长度一致
assert len(test_pred) == len(test_scores)

# 计算每个 srch_id 的 NDCG@5 分数
ndcg_scores = []

unique_ids = np.unique(test_groups)
ly = 1
for srch_id in unique_ids:
    ly += 1
    indices = [i for i, x in enumerate(test_groups) if x == srch_id]
    true_relevance = [test_scores[i] for i in indices]
    pred_relevance = [test_pred[i] for i in indices]
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [pred_relevance], k=5))
print(ly)
# 计算平均 NDCG@5 分数
average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score@5: {average_ndcg}")

# 打印模型总结
model.summary()


In [ ]:
print(len(ndcg_scores))

In [ ]:
print(test_pred)
print(len(test_pred))

# Test 5

In [ ]:
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0
    
# 逐个读取feature_engineered_training_chunk{i}并上下拼接到一个df
base_path = 'D:/Table/P5/DM-AS2/Data_featured_menghan/'
file_pattern = 'best_feature_engineered_training_chunk_{}.csv'
for i in range(2):
    df_chunk = pd.read_csv(base_path + file_pattern.format(i))
    df_chunk['score'] = df_chunk.apply(assign_scores, axis=1)
    if i == 0:
        df = df_chunk
    else:
        df = pd.concat([df, df_chunk], axis=0)
df.head()

In [ ]:
# 统计每个 srch_id 的数据条数
srch_id_counts = df['srch_id'].value_counts()

# 过滤掉数据量小于15的 srch_id
valid_srch_ids = srch_id_counts[srch_id_counts >= 15].index
filtered_df = df[df['srch_id'].isin(valid_srch_ids)]

# 对于每个 srch_id，按照 score 值排序，并保留前15条记录
processed_df = filtered_df.groupby('srch_id').apply(lambda x: x.nlargest(15, 'score')).reset_index(drop=True)

# 特征列，排除不需要的列
feature_columns = [col for col in processed_df.columns if col not in ['srch_id', 'date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd']]

# 标准化特征数据
scaler = StandardScaler()
processed_df[feature_columns] = scaler.fit_transform(processed_df[feature_columns])

# 确认处理后的数据长度一致
print(processed_df.groupby('srch_id').size().value_counts())


In [ ]:
def generator(dataframe, is_train=True):
    grouped = dataframe.groupby('srch_id')
    for name, group in grouped:
        X = group[feature_columns].values.astype(np.float32)
        if is_train:
            y = group['score'].values.astype(np.float32)
            yield X, y
        else:
            srch_id = group['srch_id'].values[0]
            yield X, srch_id

def make_dataset(dataframe, batch_size, is_train=True):
    if is_train:
        dataset = tf.data.Dataset.from_generator(
            lambda: generator(dataframe, is_train),
            output_signature=(
                tf.TensorSpec(shape=(None, len(feature_columns)), dtype=tf.float32),
                tf.TensorSpec(shape=(None,), dtype=tf.float32)
            )
        )
    else:
        dataset = tf.data.Dataset.from_generator(
            lambda: generator(dataframe, is_train),
            output_signature=(
                tf.TensorSpec(shape=(None, len(feature_columns)), dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.int64)
            )
        )
    return dataset.batch(batch_size)

# 获取唯一的查询ID
unique_ids = processed_df['srch_id'].unique()

# 划分训练集和测试集的查询ID
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42, shuffle=True)

# 根据 train_ids 划分训练集和验证集的查询ID
train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42, shuffle=True)

# 根据查询ID过滤数据
train_df = processed_df[processed_df['srch_id'].isin(train_ids)]
val_df = processed_df[processed_df['srch_id'].isin(val_ids)]
test_df = processed_df[processed_df['srch_id'].isin(test_ids)]

# 创建训练、验证和测试数据集
batch_size = 1
train_dataset = make_dataset(train_df, batch_size).shuffle(buffer_size=len(train_df['srch_id'].unique()))
val_dataset = make_dataset(val_df, batch_size).shuffle(buffer_size=len(val_df['srch_id'].unique()))
test_dataset = make_dataset(test_df, batch_size, is_train=False)


In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
import tensorflow_ranking as tfr

# 创建 ApproxNDCGLoss 对象
loss = tfr.keras.losses.ApproxNDCGLoss(
    reduction=tf.losses.Reduction.AUTO,
    lambda_weight=tfr.keras.losses.DCGLambdaWeight(topn=5),
    temperature=0.1,
    ragged=False
)

# 构建模型
def build_model(input_shape):
    input_layer = Input(shape=input_shape)
    dense_layer = Dense(256, activation='relu')(input_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(128, activation='relu')(dropout_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(64, activation='relu')(dropout_layer)
    output_layer = Dense(1, activation='linear')(dense_layer)
    output_layer = tf.squeeze(output_layer, axis=-1)  # 调整输出形状为 [batch_size, list_size]
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss)
    return model

# 构建并训练模型
input_shape = (15, len(feature_columns))
model = build_model(input_shape)
model.fit(train_dataset, epochs=5, validation_data=val_dataset)


In [ ]:
test_pred = model.predict(test_dataset)
test_pred = np.concatenate(test_pred, axis=0)
# 评估模型，计算 NDCG 分数
test_df['predictions'] = test_pred

# 首先确保数据按照 srch_id 和 predictions 降序排序
test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = test_df.groupby('srch_id')
ndcg_scores = []
ly = 1
for name, group in grouped:
    ly += 1
    group = group.sort_values('predictions', ascending=False)
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    # 计算当前搜索会话的 NDCG 分数，并追加到列表中
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))
print(ly)
average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")

# Result output 等长数据

In [ ]:
# Assume the model and all setup are correctly initialized and available as earlier described

# Reading the chunked data for prediction
chunk_size = 10000
reader = pd.read_csv('D:/Table/P5/DM-AS2/Data_featured_menghan/best_feature_engineered_test_set_VU_DM.csv', chunksize=chunk_size)
predictions = []

for chunk in reader:
    # Data preprocessing (same as training data)
    chunk[feature_columns] = scaler.transform(chunk[feature_columns])

    # Dividing each srch_id data into groups of 15 records
    grouped = chunk.groupby('srch_id')
    temp_preds = []
    for name, group in grouped:
        n = len(group)
        num_full_groups = n // 15
        remainder = n % 15

        # Predicting in full groups
        for i in range(num_full_groups):
            sub_group = group.iloc[i*15:(i+1)*15]
            sub_group_dataset = make_dataset(sub_group, batch_size=1, is_train=False)
            sub_group_pred = model.predict(sub_group_dataset)
            temp_preds.extend(zip(sub_group['srch_id'], sub_group['prop_id'], np.concatenate(sub_group_pred, axis=0)))

        # Handling the remainder data less than 15
        if remainder > 0:
            sub_group = group.iloc[num_full_groups*15:]
            padding = 15 - remainder
            # Filling the remainder with zeros or the mean of the dataset, or you might replicate the last few rows to make up numbers
            padding_data = np.tile(sub_group[feature_columns].iloc[-1].values, (padding, 1))
            sub_group_padded = pd.DataFrame(np.vstack([sub_group[feature_columns].values, padding_data]), columns=feature_columns)
            sub_group_dataset = make_dataset(sub_group_padded, batch_size=1, is_train=False)
            sub_group_pred = model.predict(sub_group_dataset)[:remainder]
            temp_preds.extend(zip(sub_group['srch_id'], sub_group['prop_id'], np.concatenate(sub_group_pred, axis=0)))

    # Collecting the predictions to a DataFrame
    temp_df = pd.DataFrame(temp_preds, columns=['srch_id', 'prop_id', 'predictions'])
    predictions.append(temp_df)

# Concatenating all batches of predictions into a final DataFrame
final_predictions = pd.concat(predictions)


In [ ]:
# 确保按照预测分数排序，如果 Kaggle 要求
final_predictions.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 选择需要的列
final_predictions = final_predictions[['srch_id', 'prop_id']]

# 保存为 CSV 文件，确保不包含索引，包含列标题
final_predictions.to_csv('train=yuan_NN_init.csv', index=False, header=True)

In [ ]:
print(final_predictions)

# Test 6 不等长数据

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
import tensorflow_ranking as tfr

# 逐个读取feature_engineered_training_chunk{i}并上下拼接到一个df
base_path = 'D:/Table/P5/DM-AS2/Data_featured_menghan/'
file_pattern = 'best_feature_engineered_training_chunk_{}.csv'
for i in range(1):
    df_chunk = pd.read_csv(base_path + file_pattern.format(i))
    df_chunk['score'] = df_chunk.apply(assign_scores, axis=1)
    if i == 0:
        df = df_chunk
    else:
        df = pd.concat([df, df_chunk], axis=0)
df.head()

# 特征列，排除不需要的列
feature_columns = [col for col in df.columns if col not in ['srch_id', 'date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd']]

# 标准化特征数据
scaler = StandardScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

# 定义生成器和数据集生成函数
def generator(dataframe, is_train=True):
    grouped = dataframe.groupby('srch_id')
    for name, group in grouped:
        X = group[feature_columns].values.astype(np.float32)
        if is_train:
            y = group['score'].values.astype(np.float32)
            yield X, y
        else:
            srch_id = group['srch_id'].values[0]
            prop_id = group['prop_id'].values[0]
            yield X, (srch_id, prop_id)

def make_dataset(dataframe, batch_size, is_train=True):
    if is_train:
        dataset = tf.data.Dataset.from_generator(
            lambda: generator(dataframe, is_train),
            output_signature=(
                tf.TensorSpec(shape=(None, len(feature_columns)), dtype=tf.float32),
                tf.TensorSpec(shape=(None,), dtype=tf.float32)
            )
        )
        padded_shapes = ([None, len(feature_columns)], [None])
    else:
        dataset = tf.data.Dataset.from_generator(
            lambda: generator(dataframe, is_train),
            output_signature=(
                tf.TensorSpec(shape=(None, len(feature_columns)), dtype=tf.float32),
                (tf.TensorSpec(shape=(), dtype=tf.int64), tf.TensorSpec(shape=(), dtype=tf.int64))
            )
        )
        padded_shapes = ([None, len(feature_columns)], ())

    return dataset.padded_batch(batch_size, padded_shapes=padded_shapes)


In [ ]:
# 获取唯一的查询ID
unique_ids = df['srch_id'].unique()

# 划分训练集和测试集的查询ID
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42, shuffle=True)

# 根据 train_ids 划分训练集和验证集的查询ID
train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42, shuffle=True)

# 根据查询ID过滤数据
train_df = df[df['srch_id'].isin(train_ids)]
val_df = df[df['srch_id'].isin(val_ids)]
test_df = df[df['srch_id'].isin(test_ids)]

# 创建训练、验证和测试数据集
batch_size = 32
train_dataset = make_dataset(train_df, batch_size).shuffle(buffer_size=len(train_df['srch_id'].unique()))
val_dataset = make_dataset(val_df, batch_size).shuffle(buffer_size=len(val_df['srch_id'].unique()))
test_dataset = make_dataset(test_df, batch_size, is_train=False)

# 创建 ApproxNDCGLoss 对象
loss = tfr.keras.losses.ApproxNDCGLoss(
    reduction=tf.losses.Reduction.AUTO,
    lambda_weight=tfr.keras.losses.DCGLambdaWeight(topn=5),
    temperature=0.1,
    ragged=False
)

# 构建模型
def build_model(input_shape):
    input_layer = Input(shape=input_shape)
    dense_layer = Dense(256, activation='relu')(input_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(128, activation='relu')(dropout_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(64, activation='relu')(dropout_layer)
    output_layer = Dense(1, activation='linear')(dense_layer)
    output_layer = tf.squeeze(output_layer, axis=-1)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss)
    return model

# 构建和训练模型
input_shape = (None, len(feature_columns))
model = build_model(input_shape)
model.fit(train_dataset, epochs=5, validation_data=val_dataset)


In [ ]:
test_pred = model.predict(test_dataset)
test_pred = np.concatenate(test_pred, axis=0)
# 评估模型，计算 NDCG 分数
test_df['predictions'] = test_pred

# 首先确保数据按照 srch_id 和 predictions 降序排序
test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = test_df.groupby('srch_id')
ndcg_scores = []

for name, group in grouped:
    group = group.sort_values('predictions', ascending=False)
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    # 计算当前搜索会话的 NDCG 分数，并追加到列表中
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")

# Test 7 

### pre

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow_ranking as tfr




In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import StandardScaler
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

def expand_to_fixed_length(group, length=50):
    if len(group) < length:
        # 创建一个填充 DataFrame，除了 srch_id 之外其他值均为0
        padding = pd.DataFrame(0, index=np.arange(length - len(group)), columns=group.columns)
        padding['srch_id'] = group['srch_id'].iloc[0]
        group = pd.concat([group, padding], ignore_index=True)
    return group

# 读取和处理数据
base_path = '/Users/eva/Documents/Study/Y1S2/DMT/assignment2/'
file_pattern = 'best_feature_engineered_training_chunk_{}.csv'
df_list = []
for i in range(1):  # 修改为实际的文件数量
    df_chunk = pd.read_csv(base_path + file_pattern.format(i))
    df_chunk['score'] = df_chunk.apply(assign_scores, axis=1)
    df_list.append(df_chunk)

# 合并所有数据
df = pd.concat(df_list, axis=0)

EXPAND_LENGTH = 45
# 扩展每个 srch_id 的数据到45条
expanded_df = df.groupby('srch_id').apply(expand_to_fixed_length, length=EXPAND_LENGTH).reset_index(drop=True)

columns = df.columns

feature_columns = [
    col for col in columns if col not in ['date_time', 'position', 'click_bool', 'booking_bool', 'score', 'srch_id']
    and 'gross_bookings_usd' not in col and 'position' not in col and col != 'price_per_person_rank_percentile'
]

# 标准化特征数据
scaler = StandardScaler()
expanded_df[feature_columns] = scaler.fit_transform(expanded_df[feature_columns])

# 填充 NaN 值为 0
expanded_df[feature_columns] = expanded_df[feature_columns].fillna(0)

# 检查填充后是否还有 NaN 值
nan_count_after_filling = expanded_df[feature_columns].isna().sum().sum()
print(f'Number of NaN values after filling: {nan_count_after_filling}')

# 确认处理后的数据长度一致
print(expanded_df.groupby('srch_id').size().value_counts())

/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_10728/3533252856.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  expanded_df = df.groupby('srch_id').apply(expand_to_fixed_length, length=EXPAND_LENGTH).reset_index(drop=True)


Number of NaN values after filling: 0
45    20058
Name: count, dtype: int64


### 数据生成和批处理

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow_ranking as tfr
def generator(dataframe):
    grouped = dataframe.groupby('srch_id')
    for name, group in grouped:
        X = group[feature_columns].values.astype(np.float32)
        y = group['score'].values.astype(np.float32)
        yield X, y

def make_dataset(dataframe, batch_size):
    dataset = tf.data.Dataset.from_generator(
        lambda: generator(dataframe),
        output_signature=(
            tf.TensorSpec(shape=(EXPAND_LENGTH, len(feature_columns)), dtype=tf.float32),
            tf.TensorSpec(shape=(EXPAND_LENGTH,), dtype=tf.float32)
        )
    )
    return dataset.batch(batch_size)

# 获取唯一的查询ID
unique_ids = expanded_df['srch_id'].unique()

# 划分训练集和测试集的查询ID
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42, shuffle=True)

# 根据 train_ids 划分训练集和验证集的查询ID
train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42, shuffle=True)

# 根据查询ID过滤数据
train_df = expanded_df[expanded_df['srch_id'].isin(train_ids)]
val_df = expanded_df[expanded_df['srch_id'].isin(val_ids)]
test_df = expanded_df[expanded_df['srch_id'].isin(test_ids)]

# 创建训练、验证和测试数据集
batch_size = 32
train_dataset = make_dataset(train_df, batch_size).shuffle(buffer_size=len(train_df['srch_id'].unique()))
val_dataset = make_dataset(val_df, batch_size).shuffle(buffer_size=len(val_df['srch_id'].unique()))
test_dataset = make_dataset(test_df, batch_size)

# 查看训练数据集中的一个批次
for X, y in train_dataset.take(1):
    X_shape = X.shape

print("Feature batch (X) shape:", X_shape)



### modeling

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout, Reshape
from tensorflow.keras.models import Model
import tensorflow_ranking as tfr

# 创建 ApproxNDCGLoss 对象
loss = tfr.keras.losses.ApproxNDCGLoss(
    reduction=tf.losses.Reduction.AUTO,
    lambda_weight=tfr.keras.losses.DCGLambdaWeight(topn=5),  # 确定学习NDCG@5
    temperature=0.1,
    ragged=False  # 不使用 ragged=True
)

# 构建模型
def build_model(input_shape):
    input_layer = Input(shape=input_shape)
    dense_layer = Dense(256, activation='relu')(input_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(128, activation='relu')(dropout_layer)
    dropout_layer = Dropout(0.4)(dense_layer)
    dense_layer = Dense(64, activation='relu')(dropout_layer)
    output_layer = Dense(1, activation='linear')(dense_layer)
    output_layer = tf.squeeze(output_layer, axis=-1)  # 调整输出形状为 [batch_size, list_size]
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss)
    return model

# 构建并训练模型
input_shape = (EXPAND_LENGTH, len(feature_columns))
model = build_model(input_shape)
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

### result test

In [ ]:
test_pred = model.predict(test_dataset)
test_pred = np.concatenate(test_pred, axis=0)
# 评估模型，计算 NDCG 分数
test_df['predictions'] = test_pred

# 首先确保数据按照 srch_id 和 predictions 降序排序
test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = test_df.groupby('srch_id')
ndcg_scores = []
ly = 1
for name, group in grouped:
    ly += 1
    group = group.sort_values('predictions', ascending=False)
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    # 计算当前搜索会话的 NDCG 分数，并追加到列表中
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))
print(ly)
average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")

# Result output 等长数据

In [ ]:
# 使用迭代器逐块读取数据
chunk_size = 10000
reader = pd.read_csv('D:/Table/P5/DM-AS2/Data_featured_menghan/best_feature_engineered_test_set_VU_DM.csv', chunksize=chunk_size)

predictions = []  # 创建一个空列表以存储每个块的预测结果

for chunk in reader:
    chunk['score'] = 0
    # 计算评分并扩展数据到固定长度
    chunk = chunk.groupby('srch_id').apply(expand_to_fixed_length, length=EXPAND_LENGTH).reset_index(drop=True)

    # 特征列，排除不需要的列
    feature_columns = [col for col in expanded_df.columns if col not in ['srch_id', 'date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd']]

    # 标准化特征数据
    scaler = StandardScaler()
    chunk[feature_columns] = scaler.fit_transform(chunk[feature_columns])

    batch_size = 32
    # 填充 NaN 值为 0
    chunk[feature_columns] = chunk[feature_columns].fillna(0)
    chunk_dataset = make_dataset(chunk, batch_size)
    
    # 应用模型进行预测
    chunk_pred = model.predict(chunk_dataset)
    chunk['predictions'] = chunk_pred.flatten()  # 将预测结果展平并添加到 DataFrame
    predictions.append(chunk[['srch_id', 'prop_id', 'predictions']])  # 仅保留需要的列


In [ ]:
# 合并所有批次的预测结果
final_predictions = pd.concat(predictions)

# 确保按照预测分数排序，如果 Kaggle 要求
final_predictions.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 选择需要的列
final_predictions = final_predictions[['srch_id', 'prop_id']]

# 保存为 CSV 文件，确保不包含索引，包含列标题
final_predictions.to_csv('train=yuan_NN_test_init.csv', index=False, header=True)

In [ ]:
print(final_predictions)